# 勾配ブースティング決定木

## Gradient Boosting Machine

## 正則化つきGBDT

$n$個の観測データがあり、$m$次元の特徴量があるとする。

$$
\mathcal{D} = \{ (x_i, y_i) \}, |D| = n, x_i \in \mathbb{R}^m, y \in \mathbb{R}
$$


勾配ブースティング決定木のモデルは次のように表される

$$
\hat{y}_i = \phi(x_i) = \sum^K_{k=1} f_k(x_i)
$$

ここで
- $f_k \in \mathcal{F}$は予測器
- $\mathcal{F} = \{ f(x) = w_{q(x)} \}$は回帰木の空間
  - $q: \mathbb{R}^m \to T$は入力データ$x$を木の各葉のインデックスに割り振る写像。$T$は各木の葉の数
  - $w \in \mathbb{R}^T$は葉の重み（weights）と呼ばれ、予測に使われた葉の出力値。予測値は$w_{q(x)}$の和となるので予測値のベースでもある


学習の際は正則化付き誤差関数

$$
\mathcal{L}(\phi) = \sum^n_{i=1} l(\hat{y}, y_i) + \sum_k \Omega(f_k)
$$

を最小化する。ここで$l$は微分可能な凸関数である誤差関数で、$\Omega$は正則化項

$$
\Omega(f) = \gamma T + \frac{1}{2} \lambda ||w||^2
$$


学習は加法的に行うため$t$番目の誤差は次のようになる。

$$
\mathcal{L}^{(t)}(\phi) = \sum^n_{i=1} l(y_i, \hat{y}^{(t-1)} + f_t(x_i)) + \Omega(f_t)
$$


### テイラー展開による近似

この誤差を二次近似したものを使うことで計算量を削減することもできることが知られている（[Friedman et al., 2000](https://citeseerx.ist.psu.edu/document?repid=rep1&type=pdf&doi=6f4493eff2531536a7aeb3fc11d62c30a8f487f6)）

$$
\mathcal{L}^{(t)} \approx 
\sum^n_{i=1} [
     l(y_i, \hat{y}^{(t-1)})
     + g_i f_t(x_i)
     + \frac{1}{2} h_i f_t^2(x_i)
]
 + \Omega(f_t)
$$


ここで

$$
g_i = \frac{ \partial l(y_i, \hat{y}^{(t-1)}) }{\partial \hat{y}^{(t-1)} }\\
h_i = \frac{ \partial^2 l(y_i, \hat{y}^{(t-1)}) }{\partial (\hat{y}^{(t-1)})^2 }
$$

定数項を省略すると

$$
\tilde{\mathcal{L}}^{(t)} =
\sum^n_{i=1} [
     g_i f_t(x_i)
     + \frac{1}{2} h_i f_t^2(x_i)
]
 + \Omega(f_t)
$$



葉$j$におけるインスタンス（サンプル）の集合を$I_j = \{i|q(x_i) = j\}$と表記すると、次のように書き換えることができる

$$
\begin{align}
\tilde{\mathcal{L}}^{(t)}
&= \sum^n_{i=1} [
     g_i f_t(x_i)
     + \frac{1}{2} h_i f_t^2(x_i)
] + \gamma T + \frac{1}{2} \lambda \sum^T_{j=1} w_j^2
\\
&= \sum^T_{j=1} [
     (\sum_{i\in I_j} g_i) w_j
     + \frac{1}{2} (\sum_{i\in I_j} h_i + \lambda ) w^2_j
] + \gamma T
\end{align}
$$

固定した木の構造$q(x)$について、葉$j$の最適な重みは

$$
w^*_j = -\frac{ \sum_{i\in I_j} g_i }{ \sum_{i \in I_j} h_i + \lambda }
$$

となる。

:::{dropdown} 導出

葉$j$についての部分だけ取り出して導関数を0とおいて整理する

$$
\frac{ \partial \tilde{\mathcal{L}}^{(t)}_j }{\partial w_j }
= \sum_{i\in I_j} g_i + (\sum_{i\in I_j} h_i + \lambda ) w_j
= 0
\\
\implies
(\sum_{i\in I_j} h_i + \lambda ) w_j
= -\sum_{i\in I_j} g_i
\\
\implies
w_j = -\frac{ \sum_{i\in I_j} g_i }{ \sum_{i\in I_j} h_i + \lambda } = w_j^*
$$
:::

L1の場合

$$
\begin{align}
\tilde{\mathcal{L}}^{(t)}
&= \sum^n_{i=1} [
     g_i f_t(x_i)
     + \frac{1}{2} h_i f_t^2(x_i)
]
+ \gamma T + \frac{1}{2} \lambda \sum^T_{j=1} w_j^2
+ \alpha \sum^T_{j=1} |w_j|
\\
&= \sum^T_{j=1} [
     (\sum_{i\in I_j} g_i) w_j
     + \frac{1}{2} (\sum_{i\in I_j} h_i + \lambda ) w^2_j
] + \gamma T
\end{align}
$$



L1の場合

$$
\begin{align}
\tilde{\mathcal{L}}^{(t)}
&= \sum^n_{i=1} [
     g_i f_t(x_i)
     + \frac{1}{2} h_i f_t^2(x_i)
]
+ \gamma T + \frac{1}{2} \lambda \sum^T_{j=1} w_j^2
+ \alpha \sum^T_{j=1} |w_j|
\\
&= \sum^T_{j=1} [
     (\sum_{i\in I_j} g_i) w_j
     + \alpha |w_j|
     + \frac{1}{2} (\sum_{i\in I_j} h_i + \lambda ) w^2_j
] + \gamma T
\end{align}
$$

$\sum^T_{j=1} [\cdot]$の内側を整理すると

$$
w^*_j = -\frac
{ \sum_{i\in I_j} g_i \pm \alpha }
{ \sum_{i \in I_j} h_i + \lambda }
$$


最適な重み$w_j^*$を誤差関数に戻すと

$$
\tilde{\mathcal{L}}^{(t)}(q)
= -\frac{1}{2} \sum^T_{j=1}
\frac{ (\sum_{i\in I_j} g_i)^2 }
{ \sum_{i\in I_j} h_i + \lambda }
+ \gamma T
$$

となり、これ木の構造$q$の品質をスコアリングする関数として使うことができる。

:::{dropdown} 導出

$$
\begin{align}
\tilde{\mathcal{L}}^{(t)}
&= \sum^T_{j=1} [
     (\sum_{i\in I_j} g_i)
     (-\frac{ \sum_{i\in I_j} g_i }{ \sum_{i \in I_j} h_i + \lambda })
     + \frac{1}{2} (\sum_{i\in I_j} h_i + \lambda ) 
     (-\frac{ \sum_{i\in I_j} g_i }{ \sum_{i \in I_j} h_i + \lambda })^2
] + \gamma T
\\
&= \sum^T_{j=1} [
     -\frac{ (\sum_{i\in I_j} g_i)^2 }{ \sum_{i \in I_j} h_i + \lambda }
     + \frac{1}{2} \frac{ (\sum_{i\in I_j} g_i)^2 }{ \sum_{i \in I_j} h_i + \lambda }
] + \gamma T
\\
&= \sum^T_{j=1} [
     - \frac{1}{2} \frac{ (\sum_{i\in I_j} g_i)^2 }{ \sum_{i \in I_j} h_i + \lambda }
] + \gamma T
\end{align}
$$
:::

## 参考

- [Schapire, R. E. (1990). The strength of weak learnability. Machine learning, 5, 197-227.](http://rob.schapire.net/papers/strengthofweak.pdf)
- [Friedman, J., Hastie, T., & Tibshirani, R. (2000). Additive logistic regression: a statistical view of boosting (with discussion and a rejoinder by the authors). The annals of statistics, 28(2), 337-407.](https://citeseerx.ist.psu.edu/document?repid=rep1&type=pdf&doi=6f4493eff2531536a7aeb3fc11d62c30a8f487f6)